In [2]:
import numpy as np
import pandas as pd 
import torch.nn as nn
import re
from bs4 import BeautifulSoup
import string
import re
import contractions
import torch
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [85]:
#importing the training data
imdb_data=pd.read_csv('/kaggle/input/big-sentiment-analysis/IMDB Dataset.csv')
print(imdb_data.shape)
imdb_data.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [86]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    clean_text = soup.get_text()
    return clean_text

text = "<p>This is an <b>example</b> text with HTML tags.</p>"
clean_text = remove_html_tags(text)
print(clean_text)

This is an example text with HTML tags.


In [88]:
def remove_punctuation(text):
    clean_text = "".join([char for char in text if char not in string.punctuation])
    return clean_text

text = "Hello! This is an example sentence."
clean_text = remove_punctuation(text)
print(clean_text)

Hello This is an example sentence


In [89]:
def remove_special_characters(text):
    clean_text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    return clean_text

text = "Hello! This is @an example #sentence."
clean_text = remove_special_characters(text)
print(clean_text)

Hello This is an example sentence


In [90]:
def normalize_text(text):
    clean_text = text.lower()
    return clean_text

text = "Hello! This is an EXAMPLE sentence."
clean_text = normalize_text(text)
print(clean_text)

hello! this is an example sentence.


In [91]:
def expand_contractions(text):
    expanded_text = contractions.fix(text)
    return expanded_text

text = "I can't believe it!"
expanded_text = expand_contractions(text)
print(expanded_text)

I cannot believe it!


In [92]:
def remove_numbers(text):
    clean_text = re.sub(r"\d+", "", text)
    return clean_text

text = "This is an example sentence with 123 numbers."
clean_text = remove_numbers(text)
print(clean_text)

This is an example sentence with  numbers.


In [93]:
def remove_extra_whitespace(text):
    clean_text = " ".join(text.split())
    return clean_text

text = "   This    is   an   example   sentence.   "
clean_text = remove_extra_whitespace(text)
print(clean_text)

This is an example sentence.


In [94]:
def preprocess_text(text):

    text = remove_html_tags(text)
    text = remove_punctuation(text)
    text = remove_special_characters(text)
    text = normalize_text(text)
    text = expand_contractions(text)
    text = remove_numbers(text)
    text = remove_extra_whitespace(text)

    return text

imdb_data['review'] = imdb_data['review'].apply(preprocess_text)

/tmp/ipykernel_28/3165369254.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [95]:
labels = imdb_data['sentiment']
# change review from text to numbers negative = 0 , positive = 1
labels[labels == 'positive'] = 1 
labels[labels == 'negative'] = 0

labels= labels.tolist()


In [97]:
# Convert labels to tensors
labels = torch.tensor(labels)

In [98]:
#Load our auto tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [99]:
# Load cleaned reviews 
review = imdb_data['review'].tolist()
# Tokenization and Padding to get the attention masks and input ids
tokenized_texts = tokenizer(review, padding='max_length',max_length=128,truncation=True, return_tensors="pt",)

In [100]:
input_ids = tokenized_texts['input_ids']

attention_masks = tokenized_texts['attention_mask']

In [101]:
# Split data into training and validation sets
input_ids_train, input_ids_val, attention_masks_train, attention_masks_val, labels_train, labels_val = train_test_split(
    input_ids, attention_masks, labels, test_size=0.2, random_state=42)

In [102]:
from torch.utils.data import TensorDataset, DataLoader
batch_size=32
train_data = TensorDataset(input_ids_train, attention_masks_train, labels_train)
train_dataloader = DataLoader(train_data, batch_size=batch_size) 

val_data = (TensorDataset(input_ids_val, attention_masks_val, labels_val))
val_dataloader = DataLoader(val_data, batch_size=batch_size)

In [103]:
# Load the bert model
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [104]:
# Move model to GPU
device_ids = [0, 1]  # IDs of the GPUs you want to use
model = model.to('cuda')
model = nn.DataParallel(model, device_ids=device_ids)

In [105]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

In [106]:
# Training loop
from tqdm import tqdm
num_epochs = 5
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    correct_predictions = 0
    total_samples = 0
    
    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    
    for batch in progress_bar:
        input_ids, attention_mask, label = batch
        optimizer.zero_grad()

        input_ids = input_ids.to('cuda')
        attention_mask = attention_mask.to('cuda')
        label = label.to('cuda')

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = criterion(logits, label)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        _, predicted_labels = torch.max(logits, dim=1)
        correct_predictions += (predicted_labels == label).sum().item()
        total_samples += label.size(0)
        
        progress_bar.set_postfix({'Loss': loss.item(), 'Accuracy': correct_predictions / total_samples})
    
    average_loss = total_loss / len(train_dataloader)
    accuracy = correct_predictions / total_samples
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}')

print('Training complete!')


Epoch 1/5, Loss: 0.3035, Accuracy: 0.8651


Epoch 2/5, Loss: 0.1738, Accuracy: 0.9335


Epoch 3/5, Loss: 0.0996, Accuracy: 0.9651


Epoch 4/5, Loss: 0.0669, Accuracy: 0.9771


Epoch 5/5, Loss: 0.0415, Accuracy: 0.9866
Training complete!


In [107]:
# Validation loop
model.eval()
correct_val_predictions = 0
total_val_samples = 0

with torch.no_grad():
    for val_batch in val_dataloader:
        val_input_ids, val_attention_mask, val_label = val_batch
        val_input_ids = val_input_ids.to('cuda')
        val_attention_mask = val_attention_mask.to('cuda')
        val_label = val_label.to('cuda')

        val_outputs = model(val_input_ids, attention_mask=val_attention_mask)
        val_logits = val_outputs.logits

        _, val_predicted_labels = torch.max(val_logits, dim=1)
        correct_val_predictions += (val_predicted_labels == val_label).sum().item()
        total_val_samples += val_label.size(0)

val_accuracy = correct_val_predictions / total_val_samples
print(f'Validation Accuracy: {val_accuracy:.4f}')

print('Training complete!')

Validation Accuracy: 0.8955
Training complete!


In [109]:
# Save the model
model.module.save_pretrained("Model")

In [110]:
# Load the saved model
model = BertForSequenceClassification.from_pretrained("/kaggle/working/Model")

In [148]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
text = "I called this company for customer support and they were incredibly helpful. I was on hold for less than a minute and the person I spoke to was friendly and knowledgeable. They were able to help me with my issue quickly and efficiently. I will definitely do business with this company again."
text = preprocess_text(text)
# Tokenize the new text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
# Get model predictions 
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
  
  # Get probabilities
probs = torch.softmax(logits, dim=1)

  # Print probabilities
print("Negative sentiment probability:", probs[0][0].item()) 
print("Positive sentiment probability:", probs[0][1].item())

# Get predicted label
predicted_label = torch.argmax(probs, dim=1).item()

  # Map labels
if predicted_label == 0:
    print("Predicted Label: Negative") 
else:
    print("Predicted Label: Positive")

Negative sentiment probability: 0.017505761235952377
Positive sentiment probability: 0.9824942946434021
Predicted Label: Positive
